<center> 
    <h1> Transformer TTS: A Text-to-Speech Transformer in TensorFlow 2 </h1>
    <h2> Audio synthesis with Autoregressive Transformer TTS and MelGAN Vocoder</h2>
</center>

## Autoregressive Model

In [2]:
# Clone the Transformer TTS and MelGAN repos
!git clone https://github.com/as-ideas/TransformerTTS.git
!git clone https://github.com/seungwonpark/melgan.git

Cloning into 'TransformerTTS'...
remote: Enumerating objects: 423, done.
remote: Counting objects: 100% (423/423), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 2825 (delta 260), reused 384 (delta 234), pack-reused 2402
Receiving objects: 100% (2825/2825), 8.01 MiB | 11.82 MiB/s, done.
Resolving deltas: 100% (1897/1897), done.
Cloning into 'melgan'...
remote: Enumerating objects: 396, done.
remote: Total 396 (delta 0), reused 0 (delta 0), pack-reused 396
Receiving objects: 100% (396/396), 18.04 MiB | 9.37 MiB/s, done.
Resolving deltas: 100% (185/185), done.


In [3]:
# Install requirements
!apt-get install -y espeak
!pip install -r TransformerTTS/requirements.txt

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 39 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak-data amd64 1.48.04+dfsg-5 [934 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libespeak1 amd64 1.48.04+dfsg-5 [145 kB]
G

In [4]:
# Download the pre-trained weights
! wget https://public-asai-dl-models.s3.eu-central-1.amazonaws.com/TransformerTTS/ljspeech_melgan_autoregressive_transformer.zip
! unzip ljspeech_melgan_autoregressive_transformer.zip

--2020-09-03 09:27:04--  https://public-asai-dl-models.s3.eu-central-1.amazonaws.com/TransformerTTS/ljspeech_melgan_autoregressive_transformer.zip
Resolving public-asai-dl-models.s3.eu-central-1.amazonaws.com (public-asai-dl-models.s3.eu-central-1.amazonaws.com)... 52.219.72.172
Connecting to public-asai-dl-models.s3.eu-central-1.amazonaws.com (public-asai-dl-models.s3.eu-central-1.amazonaws.com)|52.219.72.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 177804789 (170M) [application/zip]
Saving to: ‘ljspeech_melgan_autoregressive_transformer.zip’

ljspeech_melgan_aut 100%[===================>] 169.57M  85.0MB/s    in 2.0s    

2020-09-03 09:27:06 (85.0 MB/s) - ‘ljspeech_melgan_autoregressive_transformer.zip’ saved [177804789/177804789]

Archive:  ljspeech_melgan_autoregressive_transformer.zip
   creating: ljspeech_melgan_autoregressive_transformer/
  inflating: ljspeech_melgan_autoregressive_transformer/.DS_Store  
  inflating: __MACOSX/ljspeech_melgan_aut

In [5]:
!cd TransformerTTS/; git checkout 1c1cb03

Note: checking out '1c1cb03'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 1c1cb03 Add Audio class. WaveRNN and MelGAN compatible normalizations.


In [6]:
# Set up the paths
from pathlib import Path
MelGAN_path = 'melgan/'
TTS_path = 'TransformerTTS/'
config_path = Path('ljspeech_melgan_autoregressive_transformer/melgan')

import sys
sys.path.append(TTS_path)

In [7]:
# Load pretrained models
from utils.config_manager import ConfigManager
from utils.audio import Audio

import IPython.display as ipd

config_loader = ConfigManager(str(config_path), model_kind='autoregressive')
audio = Audio(config_loader.config)
model = config_loader.load_model(str(config_path / 'autoregressive_weights/ckpt-90'))

restored weights from ljspeech_melgan_autoregressive_transformer/melgan/autoregressive_weights/ckpt-90 at step 900000


In [8]:
# Synthesize text
sentence = 'Scientists at the CERN laboratory, say that they have discovered a new particle.'
out = model.predict(sentence)

pred text mel: 402 stop out: -1.4081320762634277Stopping


In [9]:
# Convert spectrogram to wav (with griffin lim)
wav = audio.reconstruct_waveform(out['mel'].numpy().T)
ipd.display(ipd.Audio(wav, rate=config_loader.config['sampling_rate']))

### MelGAN

In [ ]:
# Do some sys cleaning
sys.path.remove(TTS_path)
sys.modules.pop('model')

<module 'model' from 'TransformerTTS/model/__init__.py'>

In [ ]:
sys.path.append(MelGAN_path)
import torch
import numpy as np

vocoder = torch.hub.load('seungwonpark/melgan', 'melgan')
vocoder.eval()

mel = torch.tensor(out['mel'].numpy().T[np.newaxis,:,:])

Downloading: "https://github.com/seungwonpark/melgan/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/seungwonpark/melgan/releases/download/v0.3-alpha/nvidia_tacotron2_LJ11_epoch6400.pt" to /root/.cache/torch/hub/checkpoints/nvidia_tacotron2_LJ11_epoch6400.pt


In [ ]:
if torch.cuda.is_available():
    vocoder = vocoder.cuda()
    mel = mel.cuda()

with torch.no_grad():
    audio = vocoder.inference(mel)

In [ ]:
# Display audio
ipd.display(ipd.Audio(audio.cpu().numpy(), rate=22050))